In [37]:
import pandas as pd
import re
from collections import OrderedDict
from bs4 import BeautifulSoup
import pdb

Breaking down the document into individual entries

In [38]:
#converting document into a string with uniform formatting and extra stuff remvoed

def remove_bracted_continue_lines(original_str):
    found = False
    text_str = re.sub('\[[^\[\]]*?conti[nm]{1}u.*?\]', '', original_str)
    if len(original_str) != len(text_str):
        found = True
        #print(re.search('(\[[^\[\]]*?conti[nm]{1}u.*?\])', original_str).group(1))
    return text_str, found
                                  
with open('../artforum index v3.odt.extracted/content.xml', encoding="utf8") as f:
    soup = BeautifulSoup(f, 'xml')
text = soup.findAll('text:p') 
catalog = ""


debug = False
prev_found = False
for t in text:
    if prev_found == True:
        if t.text == '':
            prev_found = False
            continue
        else:
            prev_found = False
    if t.text == '':
        text_str = t.text
    elif t["text:style-name"] in ("Artforum_20_illustration", "Artforum_20_reply_2f_reference", 
                                'P29', 'P34', 'P65', 'P66', 'P67', 'P68', 'P69', 'P70', 'P71'):
        text_str, prev_found = remove_bracted_continue_lines(t.text)
        if len(text_str.strip()) == 0 and prev_found:
            continue
        text_str = "\t" + text_str
    elif t['text:style-name'] in ("P44", "P43")  :
        text_str, prev_found = remove_bracted_continue_lines(t.text)
        if len(text_str.strip()) == 0 and prev_found:
            continue
        text_str = "\n\n" + text_str
    else:
        text_str, prev_found = remove_bracted_continue_lines(t.text)
        if len(text_str.strip()) == 0 and prev_found:
            continue
    catalog += text_str + "\n"
    
catalog = catalog.replace("NOTE: Please read issue and page references as in the following example:", "") #removing troublesome note lines
catalog = catalog.replace("VI:8, 40-5 April 1968 = Volume VI (6), number 8, pages 40 through 45 April 1968", "")

#split it up into the Aphabetically indexed sections

split_on_index = re.split(r'^([A-Z]{1})$', catalog, flags=re.MULTILINE)
intro = split_on_index[0]
del split_on_index[0]
all_entries = OrderedDict(zip(split_on_index[0::2], split_on_index[1::2]))

# split each section on blank lines

all_entries_split = OrderedDict()
for idx_letter, catalog_section in all_entries.items():
    catalog_section = catalog_section.replace('\n——————————————————————————————————\n\n\n—————————————————————————————————— ', '')
    easy_split = catalog_section.split("\n\n")
    easy_split = [entry.strip() for entry in easy_split if entry.strip() != '']
    all_entries_split[idx_letter] = easy_split

taking out the reference entires

Note: there are some errors with this, a few non-reference entries get filtered out, it is probably easiest to parse these by hand

In [39]:
one_line_references = []
multi_line_references = []
all_entries_no_references = OrderedDict()
for idx_letter, entries in all_entries_split.items():
    filtered_entries = []
    for entry in entries:
        if '\n' not in entry:
            one_line_references.append(entry)
        else:
            if 'see' in entry.split('\n')[0]:
                multi_line_references.append(entry)
            else:
                filtered_entries.append(entry)
    all_entries_no_references[idx_letter] = filtered_entries

In [40]:
multi_line_references

['America, see Canada\n\tLatin America headings\n\tUnited States headings',
 'American painting, see\n\tPainting, United States\n\tPainting, United States, naive\n\tPainting, United States, recent\n\tWatercolor, United States',
 'American prints, see\n\tPrints, United States, recent',
 'American sculpture, see\n\tSculpture, United States\n\tSculpture, United States, recent',
 'Art, Argentine, see Argentina: recent art\n\tCórdoba [Argentina] Bienal',
 'Art, Dutch, see Netherlands: recent art\n\tPainting, Dutch',
 'Art, graphic, see Drawing headings\n\tPhotography headings\n\tPrints headings',
 'Art, Indian (American), see Native Americans\n\tPre-Columbian art',
 'Art, naive, see Painting, United States, naive\n\tNaive art',
 'Art, Near Eastern, see Iran\n\tIsrael\n\tSumer',
 'Art, Netherlandish, see Netherlands: recent art\n\tPainting, Dutch',
 'Art, Roman, see Etruria\n\tRome, ancient',
 'Art, Spanish, see Sculpture, Spanish\n\tSpain: recent art',
 'Art, Swedish, see Folk art, Swedish\

Things we know about parsing the 'normal' entires:

    The title starts with a name in ALL CAPITAL LETTERS possibly followed by a date.
    Exhibitions have the word Exhibition at the begininng of the line and have a volume referenced at the end of the line
    Works associated with an article of exhibtion have a tab at the start of the line
    Articles are not exhibitions and have a volume referenced at the end of the line as well
    Advertisements start with the word Advertisement

In [41]:
# the parent attirbute is a pointer to the entry, article or exhibiton which the object is associated with

class Entry(object):
    def __init__(self, title, raw):
        self.title = title
        self.raw = raw
        self.articles = []
        self.exhibitions = []
        self.advertisements = []
        self.references = [] #lines that begin with see also: or see 
        
class Article(object):
    def __init__(self, raw, parent):
        self.raw = raw
        self.parent = parent
        self.title = None
        self.author = None
        self.volume = None
        self.works = []
        
class Exhibition(object):
    def __init__(self, raw, parent):
        self.raw = raw
        self.parent = parent
        self.title = None
        self.city  = None
        self.gallery = None
        self.volume = None
        self.curator = None
        self.works = []
        
class Advertisement(object):
    def __init__(self, raw, parent):
        self.raw = raw
        self.parent = parent
        self.title = None
        self.volume = None
        
class Work(object):
    def __init__(self, raw, parent):
        self.raw = raw
        self.parent = parent
        self.title = None
        self.type = None
        self.media = None
        self.page_num = None


In [42]:
#creating all the Entry objects
entry_objects = OrderedDict()
for idx_letter, entries in all_entries_no_references.items():
    entry_object_list = []
    for entry in entries:
        title = entry.split('\n')[0]
        raw = entry[entry.find('\n')+1:]
        entry_object_list.append(Entry(title, raw))
    entry_objects[idx_letter] = entry_object_list

In [43]:
for idx_letter, entries in entry_objects.items():
    for entry in entries:
        print (entry.title)

AALTO, Alvar 1899-1949
AARTS, Jan 1871-1934
ABBATE, Niccolo dell’ 1512-1571
ABBOTT, Berenice 1898-1991
ABRAMS, Arthur B.
ABRAMS, Isaac
ABRIL, Ben
ABSTRACT Expressionism
ABSTRACT Expressionism in California
ACADEMY, The
ACKERMAN, S. R. 
ACTION Painting
ACTON, Arlo 1933-
ADAIR, Lee
ADAMS, Alice
ADAMS, Ansel 1902-1984
ADAMS, Clinton 1918-2002
ADAMS, Mark 1925-2006
ADAMS, Pat
ADAMSON, Jim
ADDISON, Kent
ADDISON, Raymond
ADNAN, Etel
ADRIAN, Dennis
AESOPUS ca. 620-560 B.C.
AFRICA
AGEE, William C.
AGOSTINELLI, Mario
AGOSTINI, Peter 1913-1993
AGOSTINI, Sergio
AIKEN, William 1934-
AIR terminals
AIRCRAFT
AIROLA, Augustin
AKAWIE, Thomas 1935-
ALBAVERA, Gerolamo
ALBERS, Josef 1888-1976
ALBERT, Michel
ALBI, Mrs. J. J.
ALBUQUERQUE, New Mexico: reviews
ALDRICH, Marion
ALDRIN, Anders
ALECHINSKY, Pierre 1927-
ALEXANDER, Paul
ALEXANDER, Peter 1939-
ALFERT, Max
ALHADEFF, Gustave
ALLAN, William
ALLEN, Boyd 1931-
ALLEN, David Young
ALLEN, Elizabeth
ALLEN, Marjorie
ALLEN, Terry
ALLEN, Virginia
ALLEY, Ronald


HUXLEY, Paul 1938-
HYDE, Philip
HYDE, Scott
HYPPOLITE, Hector 1894-1948
ICAZA, Francisco 1930-
IDE, Tom
IDRAC, [J.D.?]
IHLE, John L. 1925-
IKEGAWA, Shiro
ILLUMINATED manuscripts
ILLUSTRATIONS, book
IMAI, Norio
IMPRESSIONISM
INCUNABULA
INDIA
INDIANA, Robert 1928-
INDUSTRIAL design
INGLE, Tom 1920-
INGRES, Jean Auguste Dominique 1780-1867
INJEYAN, Dicron
INNESS, George 1825-1894
INSLEY, Will 1929-
INUIT
IPOUSTEGUY, Jean 1920-2006
IPPOLITO, Angelo 1922-
IRAN
IRELAND
IRWIN, Ben
IRWIN, Gwyther 1931-
IRWIN, Robert 1928-
ISAACS, Claire Naomi 1933-
ISAACSON, Joel
ISOZAKI, Arata
ISRAEL
ISRAEL: recent art
ITALY: twentieth-century art
ITO, Miyoko
IVEY, William
IYE, Marx
IZQUIERDO, Manuel 1925-
IZUMI, Shigeru 1922-
J. PAUL GETTY Museum, Malibu, California
JACKSON, Billy Morrow 1926-
JACKSON, William
JACKSON, William Henry 1843-1942
JACOBS, Arthur
JACOBS, David 1932-
JACOBS, Harold
JACOBS, Jessie
JACOBS, Joan
JACOBSEN, Rodger 1939-
JACOBSON, Homer
JACOBUS, John
JACQUET, Alain
JAMES, Barbara
JAMES, 

In [204]:
#fiddler to see if you can find articles through process of elimination
for idx_letter, entries in entry_objects.items():
    for entry in entries:
        lines = entry.raw.split('\n')
        for line in lines:
            #I understand what all of the following lines are
            if '\t' == line[0]:
                continue
            if 'Exhibition at' == line[:13]:
                continue
            if 'Advertisement with' == line[:18]:
                continue
            if 'see also:' in line or 'See also:' in line:
                continue
            if 'see' == line[:3]:
                continue
            if 'Note:' == line[:5]:
                continue
            if re.match(r'^\[As [a-zA-Z]*\]\w*$', line) is not None:
                continue
                
            #if it doesnt match any of the above types, it must be an article, so lets check
            volume_matches = re.match(r'.*?([IVXLCDM]+[:;].*)', line)
            #does it have a volume? if not, wtf is it
            if volume_matches is None:
                print (entry.title)
                print (line)
                print ("---")

ART criticism
appropriate headings in Subject Guide for specific styles, movements, formal problems and media
---
The recent work of Darby Bannard. K. Champa. VII:2, 62-6 October 1968
[As author] 
---
BREWSTER, Sir David 1781-1868
Imagery and illusion. J. Kline. 1 :5, 24-9 [October 1962]
---
CHICAGO: reviews
Chicago. For this department see W. Halstead.
---
COLOR
Color-field painting, see Post PainterlyAbstraction
---
CONSTRUCTIVISM
The Constructivist ethos: Russia 1913-1932 (part I). R. Hunt.
---
CONSTRUCTIVISM
The Constructivist ethos: Russia 1913-1932 (part II). R. Hunt.
---
DECORATIVE arts
Ceramics, English
---
DECORATIVE arts
Furniture
---
DECORATIVE arts
Jewelry
---
DECORATIVE arts
Medals
---
DECORATIVE arts
Metalwork
---
DECORATIVE arts
Tapestries
---
DESENBERG, Irma E.
Design, see
---
DUVENECK, Frank 1848-1919
Il: “Professor Loeltz”
---
FILM: reviews
Film. For this department see entries for the following re­viewers: T. Andersen, H. Dienstfrey, M. Farber, M. McClure
---
HARVEY,

In [44]:
def identify_article_title(line):
    matches = re.match(r'(.*?)\.(.*?)(\.[ ]*[IVXLCDM]+:\d+, \d+-\d+.*)', line)
    if matches is not None:
        return {'title': matches.group(1),'author': matches.group(2), 'volume': matches.group(3), 'works':[]}
    #trying now for article format without author
    matches = re.match(r'(.*?)\.([ ]*[IVXLCDM]+:\d+, \d+-\d+.*)', line)
    if matches is not None:
        return {'title': matches.group(1),'author': None, 'volume': matches.group(2), 'works':[]}
    else:
        print ('bad article line', line)
        return False
    
def parse_exhibition(line):
    matches = re.match(r'Exhibition at (.*?)\((.*?)\)[,\.](.*?)([IVXLCDM]+:\d+.*)', line)
    if matches is not None:
        return {'gallery': matches.group(1),
                'city': matches.group(2),
                'title_author': matches.group(3),
                'volume': matches.group(4),
                'works':[]
                }
    else:
        print ('bad exhibition line', line)
        return None
    
def parse_work(line):
    try:
        matches = re.match(r'(.*?):(.*)', line)
        type_of_work = matches.group(1)
        title = matches.group(2).strip()
        if ')' == title[len(title)-1]:
            #has a media field
            media = title[title.find('(')+1:-1]
            title = title[:title.find('(')]
        else:
            media = None
    except ValueError:
        print ('this line is un parsable using the current regex: \n', line)
        
    return {'type': type_of_work, 'title': title, 'media': media}

def parse_advertisements(line):
    matches = re.match(r'Advertisement with illustration:(.*?)([IVXLCDM]+:\d+,.*)', line)
    if matches is None:
        print ('bad ad', line)
    else:
        return {'title': matches.group(1), 'volume': matches.group(2)}

In [45]:

        
entry_objects = []

for entry in all_entries_no_references['A']:
    lines = entry.split('\n')
    entry_title = lines[0]
    cur_entry = Entry(entry_title, entry)
    last_parsed_exhibition_or_article = None
    for line in lines[1:]:
        print (line)
        if 'Exhibition' in line:
            exhibition = parse_exhibition(line)
            last_parsed_exhibition_or_article = exhibition
            cur_entry.exhibitions.append(exhibition)
            continue
        elif '\t' in line:
            if 'Reply' in line or 'reply' in line:
                print ('reply line:', line)
                continue
            else:
                work = parse_work(line)
                last_parsed_exhibition_or_article['works'].append(work)
                continue
        elif 'Advertisement with illustration' in line:
            ad = parse_advertisements(line)
            cur_entry.advertisements.append(ad)
        else:
            article_info = identify_article_title(line)
            if article_info:
                last_parsed_exhibition_or_article = article_info
                cur_entry.articles.append(article_info)
            
    entry_objects.append(cur_entry)

        

Architecture. F. G. Burger & P. Coplans. IV:8, 56-7 April 1966
	architectural plan: “Wolfsburg Cultural Center, Helsinki”
Neo-Impressionism and the dream of analysis. M. Kozloff. VI:8, 40-5 April 1968
	illustration: “Landscape with hayrick”
The future of landscape painting. G. Laderman. VII:3, 57-60 November 1968
	illustration: “Mythological landscape”
Book review. Photographers on photography, ed. by N. Lyons. Review by A. Bardo. V:7, 65-6 March 1967
Photographing a Henry Moore sculpture. III:1, 25-7 September 1964
	photograph: “Three-part reclining figure”
Exhibition at East Hampton Gal (New York), Drawings in series.” E. Wasserman. VI:2, 56-7 October 1967
Advertisement with illustration: “Visionary paintings”. V:9, 5 May 1967
Exhibition at Cowie Wilshire Gal (Los Angeles). CO [Curt Opliger]. II:7, 47 January 1964
	illustration: “Rest by the sea”
American sculpture: the situation in the fifties. W. Anderson. illustrated V:10, 60-7 Summer 1967
bad article line American sculpture: the 

AttributeError: 'NoneType' object has no attribute 'group'

In [275]:
line = ' American sculpture: the situation in the fifties. W. Anderson. illustrated V:10, 60-7 Summer 1967'
matches = re.match(r'(.*?)\.(.*?)(\.[ ]*[IVXLCDM]+:\d+, \d+-\d+.*)', line)

In [276]:
matches.group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [83]:
#writer = pd.ExcelWriter('catalog_tabulated_test.xlsx', engine='xlsxwriter')
#df.to_excel(writer, sheet_name=index_letter)
#writer.save()
first_pass = OrderedDict()

for index_letter, content in all_entries.items():
    regex_str = r'^({}[A-Z]*\b.*)$'.format(index_letter)
    entries = re.split(regex_str, content, flags=re.MULTILINE)
    title_data_list = []
    del entries[0]  ## first string returned by split is notes and blank lines
    title = None
    data = None
    for entry in entries:
        if re.match(regex_str, entry):
            title_data_list.append((title, data))
            title = entry.strip()
            data = None
        else:
            data = entry.strip()
    
    if title_data_list[0][0] is None and title_data_list[0][1] is None:
        del title_data_list[0]
    
    df = pd.DataFrame(title_data_list)
    df.columns = ['entry_title', 'entry_data']
    first_pass[index_letter] = df

In [89]:
#todo

def parse_entry_title(title_string):
    is_person = False
    is_see = False
    try:
        dates = re.search(r'(\d{4}-.*)$', title_string).group(1)
    except:
        dates = None
        
    if dates is not None:
        title = title_string[:title_string.find(dates)]
        dates = title_string[title_string.find(dates):]
    else:
        title = title_string
    
    if ',' in title:
        is_person = True
    if ' see ' in title:
        is_see = True
            
    if is_see:
        print( title, dates)

def parse_entry_data(data_string):
    datas = data.split('\n')
    for d in datas:
        if 'Exhibition' in d:
            print ("exhibition line found", d)
        else:
            if ':' in d:
                work_type = re.search(r'(.*):(.*)', d).group(1)
                work_title = re.search(r'(.*):(.*)', d).group(2)
                print ("type line found", work_type)
                print ("work title found", work_title)
            else:
                print('unknown line: ', d)
                
def parse_single_liners(data_string):
    # data strings are split on ALL CAPITAL LETTER catagories, within these there are wierd single liners:
    re.findall(r'\n\n[^\n]*\n\n', data_string)

In [108]:
easy_split[100]

'ALHADEFF, Gustave\nExhibition at Maxwell Gal (San Francisco), with E. Chavez. JM [James Monte]. II:4, 15 October 1963\n\tillustration: “Still life with lemons”'